In [ ]:
!pip install mido

In [ ]:
import tensorflow as tf
import numpy as np
from mido import MidiFile, MidiTrack, Message
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
midi_files = []
midi_directory = '/content/drive/MyDrive/MIDI FILES'  # Replace with your directory path
for filename in os.listdir(midi_directory):
    if filename.endswith('.mid'):
        midi_file = MidiFile(os.path.join(midi_directory, filename))
        midi_files.append(midi_file)

In [ ]:
all_notes = []
for midi_file in midi_files:
    notes = []
    for msg in midi_file:
        if msg.type == 'note_on':
            notes.append([msg.note, msg.velocity, msg.time])
    all_notes.extend(notes)

In [ ]:
notes = np.array(all_notes)

In [ ]:
# **3. Create TensorFlow Dataset:**
dataset = tf.data.Dataset.from_tensor_slices(notes)
dataset = dataset.batch(1).map(lambda x: tf.expand_dims(x, axis=-1))

In [ ]:
# **4. Define Feature Extraction Function:**
def extract_features(notes):
    # Extract pitch, velocity, and time features
    return notes[:, :3], notes[:, 0]

In [ ]:
dataset = dataset.map(extract_features)

In [ ]:
# **5. Define RNN Model:**
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(3)  # Output layer for pitch, velocity, and duration
])

In [ ]:
# **6. Compile the Model:**
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(dataset, epochs=30)

Epoch 1/30
156445/156445 [==============================] - 815s 5ms/step - loss: 2.8279
Epoch 2/30
156445/156445 [==============================] - 783s 5ms/step - loss: 0.3614
Epoch 3/30
156445/156445 [==============================] - 773s 5ms/step - loss: 0.3060
Epoch 4/30
156445/156445 [==============================] - 777s 5ms/step - loss: 0.2166
Epoch 5/30
156445/156445 [==============================] - 758s 5ms/step - loss: 0.1846
Epoch 6/30
156445/156445 [==============================] - 767s 5ms/step - loss: 0.1633
Epoch 7/30
156445/156445 [==============================] - 757s 5ms/step - loss: 0.1449
Epoch 8/30
156445/156445 [==============================] - 758s 5ms/step - loss: 0.1284
Epoch 9/30
153716/156445 [============================>.] - ETA: 13s - loss: 0.1269

In [ ]:
# **Generate Music:**
initial_state = tf.zeros((1, 128))

generated_notes = []
for _ in range(100):
    prediction = model.predict(initial_state)

    # Extract pitch, velocity, and duration predictions
    predicted_pitch, predicted_velocity, predicted_duration = prediction[0]

    # Clip pitch and convert to an integer in MIDI range (0-127)
    clipped_pitch = np.clip(np.round(predicted_pitch), 0, 127).astype(int)

    # Clip velocity and duration to reasonable ranges
    clipped_velocity = np.clip(np.round(predicted_velocity), 0, 127).astype(int)
    clipped_duration = np.clip(np.round(predicted_duration), 1, 500).astype(int)

    # Combine clipped pitch, velocity, and duration into a list
    generated_note = [clipped_pitch, clipped_velocity, clipped_duration]

    generated_notes.append(generated_note)

In [ ]:
# **Create and Save the MIDI File:**
midi_output = MidiFile()
track = MidiTrack()
midi_output.tracks.append(track)

for pitch, velocity, duration in generated_notes:
    # Extract scalar values from NumPy arrays
    pitch_scalar = int(np.mean(pitch))  # Taking the mean of the array
    velocity_scalar = int(np.mean(velocity))  # Taking the mean of the array

In [ ]:
# Create a "note_on" message to calculate its length before appending
note_on_message = Message('note_on', note=pitch_scalar, velocity=velocity_scalar, time=0)

In [ ]:
# Append the message to the track and get its length
track.append(note_on_message)
time = track[len(track) - 1].time  # Use the track length after adding the message

In [ ]:
 # Create and append "note_off" message with correct timing
track.append(Message('note_off', note=pitch_scalar, velocity=0, time=time + duration))

In [ ]:
# Save the generated MIDI file
midi_output.save('generated_music.mid')